In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, ChebConv
from torch_geometric.data import Data
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Receive spatial grid, time grid, and initial value calculation function
def build_gcn_data(x_grid, t_grid, u_initial_func):
    nodes = []
    for tj in t_grid:
        for xi in x_grid:
            # Assign a value at the initial moment and initialize it to 0 at other times
            u0 = u_initial_func(xi) if tj == 0 else 0.0  
            nodes.append([xi, tj, u0])
    nodes = torch.tensor(nodes, dtype=torch.float32)

    # Construct spatial adjacent edges, bidirectionally linear
    edge_index = []
    num_x = len(x_grid)

    for t_step in range(len(t_grid)):
        for i in range(num_x):
            idx = t_step * num_x + i
            if i > 0:
                edge_index.append([idx, idx-1])
            if i < num_x - 1:
                edge_index.append([idx, idx+1])
                
    # Build a time propagation edge, one-way 
    # (information can only flow from past time steps to future time steps, but not in the reverse direction)
    for t_step in range(len(t_grid) - 1):
        for i in range(num_x):
            src = t_step * num_x + i
            dst = (t_step + 1) * num_x + i
            edge_index.append([src, dst])
            # edge_index.append([dst, src])
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    return Data(x=nodes, edge_index=edge_index)

In [ ]:
class BurgersGCN(nn.Module):
    def __init__(self, node_feat_dim=3, hidden_dim=25, output_dim=1 , Struct_num=1):
        super(BurgersGCN, self).__init__()
        # Adaptive Parameters
        self.s_pde = torch.nn.Parameter(torch.tensor(0.0))
        self.s_bc  = torch.nn.Parameter(torch.tensor(0.0))
        self.s_ic  = torch.nn.Parameter(torch.tensor(0.0))

        # Store the GNNConv layer
        self.gnn_layers = nn.ModuleList()  
        # Store the nn.Linear layer
        self.linear_layers = nn.ModuleList()  
        self.gnn_layers.append(GATConv(node_feat_dim,hidden_dim,heads=1,concat=False) )
        for _ in range(Struct_num):
            self.gnn_layers.append(GATConv(hidden_dim, hidden_dim,heads=1,concat=False) )
        self.linear_layers.append(nn.Linear(hidden_dim, output_dim))

        # Graph neural layer initialization weights ("robust")
        for layer in self.gnn_layers:
            d_in = layer.in_channels
            d_out = layer.out_channels
            if d_in >= d_out:
                matrix = np.eye(d_in, d_out).T  # shape [d_out, d_in]
            else:
                matrix = np.zeros((d_out, d_in))
                for j in range(min(d_out, d_in)):
                    matrix[j, j] = 1
            std = 0.085 / np.sqrt(d_out)
            noise = np.random.normal(0, std, size=(d_out, d_in))
            matrix = matrix + noise
            weight_data = torch.from_numpy(matrix).float()
            
        # Linear layer initialization weights ("robust")
        for layer in self.linear_layers:
            d_in = layer.in_features
            d_out = layer.out_features
            if d_in >= d_out:
                matrix = np.eye(d_in, d_out).T  # [d_out, d_in]
            else:
                matrix = np.zeros((d_out, d_in))
                for j in range(min(d_out, d_in)):
                    matrix[j, j] = 1
            std = 0.085 / np.sqrt(d_out)
            noise = np.random.normal(0, std, size=(d_out, d_in))
            matrix = matrix + noise
            weight_data = torch.from_numpy(matrix).float()
            layer.weight.data.copy_(weight_data)

    def forward(self, data):
            x, edge_index = data.x, data.edge_index
            h = torch.tanh(self.gnn_layers[0](x, edge_index))
            
            for i in range(Struct_num - 2):
                h =  torch.tanh(self.gnn_layers[i + 1](h, edge_index))
            
            delta_u = self.linear_layers[-1](h)  

            return data.x[:, 2:3] + delta_u
        

In [ ]:
def selfadapt_compute_loss(model, data, nu, t_grid, x_grid, loss_params):
"""
Calculate the loss of the Allen_Cahn equation, which includes adaptive loss weights. 
Parameters:
model: Neural network model, which takes data as input and outputs predicted values.
data: Input data, where the second column of data.x represents the time coordinate.
nu: Diffusion coefficient in the Allen_Cahn equation.
t_grid: Time discretization grid (1D tensor or list).
x_grid: Spatial discretization grid (1D tensor or list).
loss_params: Dictionary, containing logarithmic parameters for adaptive loss weights.
Fields 's_pde', 's_bc', 's_ic' are all torch.nn.Parameter, and their initial values are generally set to 0.0. 
Return:
total_loss: A scalar tensor representing the total loss, which is the sum of the losses from the PDE, 
            boundary conditions, and initial conditions. 
"""
    # Define the boundary value function and the initial condition function
    values = {
        'left': lambda x: torch.zeros_like(x),                
        'right': lambda x: torch.zeros_like(x),               
        'initial': lambda x: torch.sin(torch.pi * x)         
    }
    
    u_pred = model(data).squeeze()  # Shape: [num_nodes]
    
    num_t = len(t_grid)
    num_x = len(x_grid)
    u = u_pred.view(num_t, num_x)
    
    delta_t = t_grid[1] - t_grid[0]
    delta_x = x_grid[1] - x_grid[0]
    
    u_interior = u[:-1, 1:-1]  # [num_t-1, num_x-2]
    u_future   = u[1:, 1:-1]   # [num_t-1, num_x-2]
    u_left     = u[:-1, :-2]   # [num_t-1, num_x-2]
    u_right    = u[:-1, 2:]    # [num_t-1, num_x-2]
    
    u_t  = (u_future - u_interior) / delta_t                  # 时间方向前向差分
    u_x  = (u_right - u_left) / (2 * delta_x)                 # 空间方向中心差分
    u_xx = (u_right - 2 * u_interior + u_left) / (delta_x ** 2)  # 空间二阶导数
    
    pde_residual = u_t - nu * u_xx 
    loss_pde = torch.mean(pde_residual ** 2)
    
    pred_left = u[:, :1] 
    pred_right = u[:, -1:] 

    u_left_true = values['left'](pred_left)
    u_right_true = values['right'](pred_right)

    loss_bc_left = F.mse_loss(pred_left, u_left_true)
    loss_bc_right = F.mse_loss(pred_right, u_right_true)
    loss_bc = (loss_bc_left + loss_bc_right) / 2
    

    t_coords = data.x[:, 1]
    mask_initial = torch.isclose(t_coords, torch.tensor(0.0, device=t_coords.device), atol=1e-6)
    u_initial_pred = u_pred[mask_initial]

    u_initial_true = values['initial'](x_grid)
    loss_ic = F.mse_loss(u_initial_pred, u_initial_true)
    
    # L_total = exp(-s_pde)*loss_pde + exp(-s_bc)*loss_bc + exp(-s_ic)*loss_ic + (s_pde + s_bc + s_ic)
    total_loss = (torch.exp(-loss_params['s_pde']) * loss_pde +
                  torch.exp(-loss_params['s_bc']) * loss_bc +
                  torch.exp(-loss_params['s_ic']) * loss_ic +
                  (loss_params['s_pde'] + loss_params['s_bc'] + loss_params['s_ic']))
    return total_loss


In [ ]:
# Setup
x_grid = torch.linspace(-1, 1, 256)  # Spatial grid
t_grid = torch.linspace(0, 1, 100)  # Temporal grid
data = build_gcn_data(x_grid, t_grid, lambda x: torch.sin(torch.pi * x))  # Example initial condition
Struct_num = 0 
model = BurgersGCN(node_feat_dim=3, hidden_dim=23, output_dim=1 , Struct_num=Struct_num)
loss_params = {
    's_pde': model.s_pde,
    's_bc': model.s_bc,
    's_ic': model.s_ic
}
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
nu = 0.01  # Diffusion coefficient

losses = []
for epoch in range(1000):
    optimizer.zero_grad()
    loss = selfadapt_compute_loss(model, data, nu, t_grid, x_grid,loss_params)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    losses.append(loss.item())
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

    
np.savetxt("./result/Diffusion_PhyGAT/losses.txt", losses)
losses = losses[100:]
# Plot the training loss curve
plt.plot(range(len(losses)), losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
u_pred = model(data)

num_t = len(t_grid) #100
num_x = len(x_grid) #256
u_pred = u_pred.view(num_t,num_x)  
u_pred = torch.flip(torch.transpose(u_pred, 0, 1), dims=[0])
u_pred = u_pred.detach().numpy()
u_pred = u_pred[::-1, :]
# Load the dataset
burgers_sol = np.load('./data/diffusion.npz')
usol = burgers_sol['solution']  # Shape: (256, 100)
usol_flat = usol.flatten()     
u_pred_flat = u_pred.flatten()  

mse = np.mean((usol_flat - u_pred_flat) ** 2)

# Compute the difference with the true solution
diff = np.abs(usol - u_pred)

# Visualize the difference
plt.figure(figsize=(6, 5))
plt.imshow(diff, extent=[0, 1, -1, 1], origin='lower', aspect='auto', cmap='jet', vmin=0, vmax=0.5)
plt.colorbar()
plt.xlabel('$t$', size=15)
plt.ylabel('$x$', size=15)
plt.tight_layout()
plt.show()

print(f"Mean Squared Error (MSE): {mse}")

In [ ]:
# Compute the difference with the true solution
pred_val = u_pred
np.save("./result/Diffusion_PhyGAT/pred_val.npy", pred_val)
print(f"预测值")
# Visualize the difference
plt.figure(figsize=(6, 5))
plt.imshow(pred_val, extent=[0, 1, -1, 1], origin='lower', aspect='auto', cmap='jet', vmin=-1, vmax=1)
plt.colorbar()
plt.xlabel('$t$', size=15)
plt.ylabel('$x$', size=15)
plt.tight_layout()
plt.show()


In [ ]:
print(f"真实值")
# Compute the difference with the true solution
true_val = usol

# Visualize the difference
plt.figure(figsize=(6, 5))
plt.imshow(true_val, extent=[0, 1, -1, 1], origin='lower', aspect='auto', cmap='jet', vmin=-1, vmax=1)
plt.colorbar()
plt.xlabel('$t$', size=15)
plt.ylabel('$x$', size=15)
plt.tight_layout()
plt.show()

In [ ]:
import time
import tracemalloc

In [ ]:
tracemalloc.start()

u_pred=model(data)
start_time = time.time() 


for i in range(100):
    u_pred=model(data)

end_time = time.time()  

print(f"运行时间: {end_time - start_time:.6f} 秒")
current, peak = tracemalloc.get_traced_memory()
print(f"当前内存使用: {current / 1024 / 1024:.2f} MB; 峰值: {peak / 1024 / 1024:.2f} MB")
tracemalloc.stop()